In [6]:
import sqlite3
import pandas as pd
import re

def get_relations(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get the create table statements
    cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    relations = {}

    for table_name, sql in tables:
        # Find references to other tables
        matches = re.findall('FOREIGN KEY\s*?\((.*?)\)\s*?REFERENCES\s*?(.*?)\(', sql)

        for match in matches:
            referenced_table = match[1].strip()
            if referenced_table not in relations:
                relations[referenced_table] = []
            relations[referenced_table].append(table_name)

    return relations

# Get the relations
db_path = 'chinook.db'
relations = get_relations(db_path)

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Load the data from the related tables into pandas DataFrames
dataframes = {}
for table_name in relations.keys():
    dataframes[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

df = pd.DataFrame(list(relations.items()), columns=['Table', 'Referenced by'])
df

,Table,Referenced by
0,"""artists""",[albums]
1,"""employees""","[customers, employees]"
2,"""customers""",[invoices]
3,"""invoices""",[invoice_items]
4,"""tracks""","[invoice_items, playlist_track]"
5,"""playlists""",[playlist_track]
6,"""albums""",[tracks]
7,"""genres""",[tracks]
8,"""media_types""",[tracks]
